In [1]:
import os
from dotenv import load_dotenv
import datetime as dt
from datetime import date
from datetime import timedelta
import pandas as pd
import numpy as np

In [2]:
load_dotenv()
my_api_key = os.getenv("API_KEY")
my_api_secret = os.getenv("API_SECRET")

# print(my_api_key)
# print(my_api_secret)

In [3]:
params = {
    'SYMBOL' : 'BTCUSDT'
    , 'FROM_TIME' : dt.datetime(2021, 1, 1) # (year, month, day, hour, minutes, seconds)
    , 'TO_TIME' : dt.datetime(2021, 11, 30)
    , 'INTERVAL' : 'D' # Possible Values: 1 3 5 15 30 60 120 240 360 720 "D" "W"
    , 'TRADE_AMOUNT' : float(10000) # Amount used per trade. Remains constant throughout time
    , 'TAKE_PROFIT_PCT' : float(2) # 2%
    , 'STOP_LOSS_PCT' : float(1)   # 1%
    , 'FEES_PCT' : float(0.075) # 0.075%
    
    , 'MIN_DATA_SIZE' : 201 # Cannot run Strategy on data set less than this value
    , 'HISTORICAL_FILES_PATH' : 'ByBit Historical' # Folder location where to store ByBit original raw data
    , 'RESULTS_PATH' : 'BackTesting Results' # Folder location where to store the back testing output
    , 'OUPUT_FILE_FORMAT' : ['xlsx']  # Prefered format(s) for the ouput: csv, xlsx or both
    
    , 'UTC_OFFSET' : -5
}

# Adjust from_time to also include 200 additional prior entries (for ema200)
# and also take into consideration the UTC offset
def adjust_from_time(from_time, interval):
    
    delta = 200
    
    # Possible Values: 1 3 5 15 30 60 120 240 360 720 "D" "W" 
    if interval not in [1, 3, 5, 15, 30, 60, 120, 240, 360, 720, "D", "W"]:
        return from_time
    
    if interval == 'W':
        result = from_time - timedelta(weeks=delta)
    elif interval == 'D':
        result = from_time - timedelta(days=delta)
    else:
        result = from_time - timedelta(minutes=interval*delta)
    return result

In [4]:
from pybit import HTTP

# Authenticated
session_auth = HTTP(
    endpoint = 'https://api.bybit.com',
    api_key = my_api_key,
    api_secret = my_api_secret
)



# The issue with ByBit API is that you can get a maximum of 200 bars from it. 
# So if you need to get data for a large portion of the time you have to call it multiple times.


print(f'Fetching data from ByBit.') 
df_list = []

# Adjust from time to also include 200 additional prior entries (for ema200)
adjusted_from_time = adjust_from_time(params['FROM_TIME'], params['INTERVAL'])

last_datetime_stamp = adjusted_from_time.timestamp()
to_time = params['TO_TIME'].timestamp() 

while last_datetime_stamp < to_time:
    print(f'Fetching next 200 lines fromTime: last_datetime: {last_datetime_stamp} < to_time: {to_time}')
    print(f'Fetching next 200 lines fromTime: last_datetime: {dt.datetime.fromtimestamp(last_datetime_stamp)} < to_time: {dt.datetime.fromtimestamp(to_time)}')
    result = session_auth.query_kline(symbol=params['SYMBOL'], interval=params['INTERVAL'], **{'from':last_datetime_stamp})['result']
    tmp_df = pd.DataFrame(result)

    if tmp_df is None or (len(tmp_df.index) == 0):
        break

    tmp_df.index = [dt.datetime.fromtimestamp(x) for x in tmp_df.open_time]
    #tmp_df.index = [dt.datetime.utcfromtimestamp(x) for x in tmp_df.open_time]
    df_list.append(tmp_df)
    last_datetime_stamp = float(max(tmp_df.open_time) + 1) # Add 1 sec to last data received

    #time.sleep(2) # Sleep for x seconds, to avoid being locked out
 
df = pd.concat(df_list)

# # Drop rows that have a timestamp greater than to_time
df = df[df.open_time <= int(params['TO_TIME'].timestamp())]

df

Fetching data from ByBit.
Fetching next 200 lines fromTime: last_datetime: 1592193600.0 < to_time: 1638248400.0
Fetching next 200 lines fromTime: last_datetime: 2020-06-15 00:00:00 < to_time: 2021-11-30 00:00:00
Fetching next 200 lines fromTime: last_datetime: 1609459201.0 < to_time: 1638248400.0
Fetching next 200 lines fromTime: last_datetime: 2020-12-31 19:00:01 < to_time: 2021-11-30 00:00:00
Fetching next 200 lines fromTime: last_datetime: 1626739201.0 < to_time: 1638248400.0
Fetching next 200 lines fromTime: last_datetime: 2021-07-19 20:00:01 < to_time: 2021-11-30 00:00:00
Fetching next 200 lines fromTime: last_datetime: 1638144001.0 < to_time: 1638248400.0
Fetching next 200 lines fromTime: last_datetime: 2021-11-28 19:00:01 < to_time: 2021-11-30 00:00:00


,id,symbol,period,interval,start_at,open_time,volume,open,high,low,close,turnover
2020-06-15 20:00:00,368951,BTCUSDT,D,D,1592265600,1592265600,6049.665,9418.0,9584.5,9372.5,9521.0,5.759886e+07
2020-06-16 20:00:00,373518,BTCUSDT,D,D,1592352000,1592352000,6573.471,9521.0,9564.0,9243.0,9462.0,6.219818e+07
2020-06-17 20:00:00,377843,BTCUSDT,D,D,1592438400,1592438400,4392.173,9462.0,9485.5,9277.0,9383.0,4.121176e+07
2020-06-18 20:00:00,382131,BTCUSDT,D,D,1592524800,1592524800,7772.374,9383.0,9439.5,9241.0,9306.5,7.233360e+07
2020-06-19 20:00:00,386359,BTCUSDT,D,D,1592611200,1592611200,6326.850,9306.5,9392.0,9176.5,9351.0,5.916237e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-24 19:00:00,19455593,BTCUSDT,D,D,1637798400,1637798400,24965.578,57185.5,59440.5,57041.5,58937.0,1.455887e+09
2021-11-25 19:00:00,19621811,BTCUSDT,D,D,1637884800,1637884800,51957.556,58937.0,59150.0,53563.0,53800.5,2.873726e+09
2021-11-26 19:00:00,19795397,BTCUSDT,D,D,1637971200,1637971200,22101.393,53800.5,55298.5,53684.5,54778.5,1.208036e+09
2021-11-27 19:00:00,19973651,BTCUSDT,D,D,1638057600,1638057600,29634.798,54778.5,57435.5,53122.0,57299.0,1.630099e+09
